In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1.0/255, rotation_range=30, horizontal_flip=True)
train_generator = datagen.flow_from_directory('Dataset/Training', target_size=(224, 224), batch_size=32)
train_generator_bin = datagen.flow_from_directory('Dataset_bin/Training', target_size=(224, 224), batch_size=32)

Found 5712 images belonging to 4 classes.
Found 5712 images belonging to 2 classes.


In [7]:
import os

# Define dataset directories
train_dir = "D:\Brain tumour classification\Dataset_bin\Training"
val_dir = "D:\Brain tumour classification\Dataset_bin\Validation"
test_dir = "D:\Brain tumour classification\Dataset_bin\Test"

# Function to count images in each class
def count_images(directory):
    classes = os.listdir(directory)
    counts = {cls: len(os.listdir(os.path.join(directory, cls))) for cls in classes}
    return counts

# Display dataset distribution
print("Training Data Distribution:", count_images(train_dir))
print("Validation Data Distribution:", count_images(val_dir))
print("Test Data Distribution:", count_images(test_dir))


Training Data Distribution: {'No_Tumor': 1595, 'Tumor': 4117}
Validation Data Distribution: {'No_Tumor': 405, 'Tumor': 4117}
Test Data Distribution: {'No_Tumor': 405, 'Tumor': 906}


In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0

# Base model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification: Tumor or No Tumor

# Detection model
detection_model = Model(inputs=base_model.input, outputs=output)
detection_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

detection_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling (Rescaling)         │ (None, 224, 224, 3)       │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization (Normalization) │ (None, 224, 224, 3)       │               7 │ rescaling[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_1 (Rescaling)       │ (None, 224, 224, 3)       │               0 │ normalization[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 225, 225, 3)       │               0 │ rescaling_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 112, 112, 32)      │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 112, 112, 32)      │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 112, 112, 32)      │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 112, 112, 32)      │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 112, 112, 32)      │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 112, 112, 32)      │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             264 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 32)          │             288 │ block1a_se_reduce[0][0]    │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 4,377,764 (16.70 MB)

 Trainable params: 4,335,741 (16.54 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Dataset directories
train_dir = "D:\Brain tumour classification\Dataset_bin\Training"
val_dir = "D:\Brain tumour classification\Dataset_bin\Validation"
test_dir = "D:\Brain tumour classification\Dataset_bin\Test"

# Data generators
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # Binary classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Detection model
detection_model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile model
detection_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


Found 5712 images belonging to 2 classes.
Found 4522 images belonging to 2 classes.
Found 1311 images belonging to 2 classes.


In [6]:
# Train the model
detection_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
)

# Evaluate the model
results = detection_model.evaluate(test_generator)
print(f"Tumor Detection Model Accuracy: {results[1] * 100:.2f}%")


Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 703s 4s/step - accuracy: 0.9524 - loss: 0.1319 - val_accuracy: 0.0896 - val_loss: 3.2722
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 624s 3s/step - accuracy: 0.9870 - loss: 0.0355 - val_accuracy: 0.0896 - val_loss: 2.5175
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 678s 4s/step - accuracy: 0.9903 - loss: 0.0324 - val_accuracy: 0.0896 - val_loss: 4.1724
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 683s 4s/step - accuracy: 0.9928 - loss: 0.0193 - val_accuracy: 0.9184 - val_loss: 2.9201
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 704s 4s/step - accuracy: 0.9979 - loss: 0.0100 - val_accuracy: 0.0896 - val_loss: 4.8321
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 710s 4s/step - accuracy: 0.9954 - loss: 0.0196 - val_accuracy: 0.9104 - val_loss: 825.2162
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 710s 4s/step - accuracy: 0.9939 - loss: 0.0156 - val_accuracy: 0.9104 - val_loss: 229.3325
Epoch 8/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 722s 4s/step - accuracy: 0.9993 - loss: 0.0045 - val_

In [10]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0

# Base model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(4, activation='softmax')(x)
classification_model = Model(inputs=base_model.input, outputs=output) # 4 classes: No Tumor + 3 tumor types

# Build model
classification_model = Model(inputs=base_model.input, outputs=output)
classification_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

classification_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_2 (Rescaling)       │ (None, 224, 224, 3)       │               0 │ input_layer_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization_1               │ (None, 224, 224, 3)       │               7 │ rescaling_2[0][0]          │
│ (Normalization)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_3 (Rescaling)       │ (None, 224, 224, 3)       │               0 │ normalization_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 225, 225, 3)       │               0 │ rescaling_3[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 112, 112, 32)      │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 112, 112, 32)      │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 112, 112, 32)      │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 112, 112, 32)      │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 112, 112, 32)      │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 112, 112, 32)      │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             264 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 32)          │             28

 Total params: 4,378,535 (16.70 MB)

 Trainable params: 4,336,512 (16.54 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os

train_dir = 'dataset/Training'
val_dir = 'dataset/Validation'
test_dir = 'dataset/Test'

input_size = (224, 224)
batch_size = 32
num_classes = 4

# Data Generators
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=input_size, batch_size=batch_size, class_mode='categorical'
)
val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=input_size, batch_size=batch_size, class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=input_size, batch_size=batch_size, class_mode='categorical', shuffle=False
)

print(train_generator.class_indices)
batch_images, batch_labels = next(iter(train_generator))
print(f"Batch images shape: {batch_images.shape}")
print(f"Batch labels shape: {batch_labels.shape}")

# Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()




Found 5712 images belonging to 4 classes.
Found 4522 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
{'Glioma': 0, 'Meningioma': 1, 'No_Tumor': 2, 'Pituitary': 3}
Batch images shape: (32, 224, 224, 3)
Batch labels shape: (32, 4)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,780,868 (56.38 MB)

 Trainable params: 66,180 (258.52 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [17]:
# Train
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

# Evaluate
results = model.evaluate(test_generator)
print(f"Tumor Classification Model Accuracy: {results[1] * 100:.2f}%")


Epoch 1/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 2846s 16s/step - accuracy: 0.2934 - loss: 1.4666 - val_accuracy: 0.6172 - val_loss: 1.2407
Epoch 2/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 1407s 8s/step - accuracy: 0.4970 - loss: 1.1790 - val_accuracy: 0.7353 - val_loss: 1.0967
Epoch 3/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 1403s 8s/step - accuracy: 0.6289 - loss: 1.0401 - val_accuracy: 0.7437 - val_loss: 1.0085
Epoch 4/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 1401s 8s/step - accuracy: 0.6856 - loss: 0.9345 - val_accuracy: 0.7636 - val_loss: 0.9215
Epoch 5/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 1401s 8s/step - accuracy: 0.7246 - loss: 0.8607 - val_accuracy: 0.7711 - val_loss: 0.8523
41/41 ━━━━━━━━━━━━━━━━━━━━ 184s 4s/step - accuracy: 0.6903 - loss: 0.9109
Tumor Classification Model Accuracy: 74.07%


In [18]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_true = test_generator.classes
y_pred = classification_model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

# Classification report
target_names = list(test_generator.class_indices.keys())  # Get class names
print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=target_names))

41/41 ━━━━━━━━━━━━━━━━━━━━ 587s 14s/step
Confusion Matrix:
[[ 49 251   0   0]
 [  6 300   0   0]
 [  7 388  10   0]
 [  0 300   0   0]]
Classification Report:
              precision    recall  f1-score   support

      Glioma       0.79      0.16      0.27       300
  Meningioma       0.24      0.98      0.39       306
    No_Tumor       1.00      0.02      0.05       405
   Pituitary       0.00      0.00      0.00       300

    accuracy                           0.27      1311
   macro avg       0.51      0.29      0.18      1311
weighted avg       0.55      0.27      0.17      1311



C:\Users\ganes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ganes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ganes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [26]:
detection_model.save('detection_model.h5')
classification_model.save('classification_model.h5')


In [23]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def predict(image_path, detection_model, classification_model):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Tumor detection
    if detection_model.predict(img_array) > 0.3:
        print("Tumor detected!")
        class_prediction = classification_model.predict(img_array)
        tumor_type = np.argmax(class_prediction)
        print(f"Tumor Type: {tumor_type}")
    else:
        print("No Tumor detected.")
        
predict("D:\Brain tumour classification\Dataset\Test\Meningioma\Te-me_0036.jpg",detection_model, classification_model)
predict("D:\Brain tumour classification\Dataset\Validation\Pituitary\Tr-pi_0112.jpg",detection_model, classification_model)
predict("D:\Brain tumour classification\Dataset\Training\Pituitary\Tr-pi_0036.jpg",detection_model, classification_model)
predict("D:\Brain tumour classification\Dataset\Training\Pituitary\Tr-pi_0019.jpg",detection_model, classification_model)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
Tumor detected!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Tumor Type: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
No Tumor detected.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
No Tumor detected.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
Tumor detected!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Tumor Type: 2


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 units for 4 classes
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Appropriate loss for multi-class
    metrics=['accuracy']
)

model.summary()

C:\Users\ganes\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │      23,888,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,907,908 (91.20 MB)

 Trainable params: 23,907,908 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    'dataset/Training',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'dataset/Validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'dataset/Test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 5712 images belonging to 4 classes.
Found 4522 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [105]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 329s 2s/step - accuracy: 0.6065 - loss: 1.2586 - val_accuracy: 0.9045 - val_loss: 0.2656
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 380s 2s/step - accuracy: 0.8949 - loss: 0.2987 - val_accuracy: 0.9584 - val_loss: 0.1452
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 341s 2s/step - accuracy: 0.9414 - loss: 0.1715 - val_accuracy: 0.9710 - val_loss: 0.0787
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 313s 2s/step - accuracy: 0.9599 - loss: 0.1162 - val_accuracy: 0.9839 - val_loss: 0.0509
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 310s 2s/step - accuracy: 0.9668 - loss: 0.0908 - val_accuracy: 0.9889 - val_loss: 0.0315
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 325s 2s/step - accuracy: 0.9770 - loss: 0.0695 - val_accuracy: 0.9967 - val_loss: 0.0144
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step - accuracy: 0.9832 - loss: 0.0426 - val_accuracy: 0.9982 - val_loss: 0.0107
Epoch 8/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 312s 2s/step - accuracy: 0.9846 - loss: 0.0415 - val_accu

In [106]:
results = model.evaluate(test_generator)
print("Detection Accuracy: {:.2f}%".format(results[1] * 100))
print("Classification Accuracy: {:.2f}%".format(results[1] * 100))

41/41 ━━━━━━━━━━━━━━━━━━━━ 8s 191ms/step - accuracy: 0.9292 - loss: 0.2813
Detection Accuracy: 95.27%
Classification Accuracy: 95.27%


In [21]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def predict(image_path, detection_model, classification_model):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Tumor detection
    if detection_model.predict(img_array) > 0.3:
        print("Tumor detected!")
        class_prediction = classification_model.predict(img_array)
        tumor_type = np.argmax(class_prediction)
        print(f"Tumor Type: {tumor_type}")
    else:
        print("No Tumor detected.")
        
predict("D:\Brain tumour classification\Dataset\Test\Meningioma\Te-me_0036.jpg",detection_model, classification_model)
predict("D:\Brain tumour classification\Dataset\Validation\Pituitary\Tr-pi_0112.jpg",detection_model, classification_model)
predict("D:\Brain tumour classification\Dataset\Training\Meningioma\Tr-me_0035.jpg",detection_model, classification_model)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
Tumor detected!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Tumor Type: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
No Tumor detected.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
Tumor detected!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Tumor Type: 2
